# Spark Structured Streaming - Demo
## Fire alarm

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
import io
from pyspark.sql.functions import *
import time
import json
import struct
import requests 

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.5,org.apache.kafka:kafka-clients:2.6.0 pyspark-shell'
                                    
spark = (SparkSession.builder 
    .master("local[*]")
    .appName("test")
    .getOrCreate()
        )

spark

set up the environment variables

In [2]:
smoke_topic = 'SmokeSensorEvent'
temperature_topic = 'TemperatureSensorEvent'
servers = "kafka:9092"

## Understanding spark-kafka integration
Let's treat first kafka as a bulk source

In [3]:
smoke_df = (spark
  .read
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("subscribe", smoke_topic)
  .option("startingOffsets", "earliest")
  .option("endingOffsets", "latest")
  .load())

In [4]:
smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [5]:
smoke_df.show(5)

+-------+--------------------+----------------+---------+------+--------------------+-------------+
|    key|               value|           topic|partition|offset|           timestamp|timestampType|
+-------+--------------------+----------------+---------+------+--------------------+-------------+
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     0|2021-10-18 10:32:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     1|2021-10-18 10:32:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     2|2021-10-18 10:32:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     3|2021-10-18 10:32:...|            0|
|[53 31]|[7B 22 73 65 6E 7...|SmokeSensorEvent|        0|     4|2021-10-18 10:33:...|            0|
+-------+--------------------+----------------+---------+------+--------------------+-------------+
only showing top 5 rows



In [6]:
stringified_smoke_df = smoke_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
stringified_smoke_df.show(5,False)

+---+--------------------------------------------------+
|key|value                                             |
+---+--------------------------------------------------+
|S1 |{"sensor": "S1", "smoke": false, "ts": 1634553145}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1634553156}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1634553166}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1634553176}|
|S1 |{"sensor": "S1", "smoke": false, "ts": 1634553186}|
+---+--------------------------------------------------+
only showing top 5 rows



In [10]:
stringified_smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



In [7]:
from pyspark.sql.types import *

smoke_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("smoke", BooleanType(), True),
    StructField("ts", TimestampType(), True)])

In [8]:
smoke_df = stringified_smoke_df.select(col("key").cast("string"),from_json(col("value"), smoke_schema).alias("value"))

In [9]:
smoke_df.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- sensor: string (nullable = true)
 |    |-- smoke: boolean (nullable = true)
 |    |-- ts: timestamp (nullable = true)



In [11]:
smoke_df.select("value.*").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2021-10-18 10:32:25|
|    S1|false|2021-10-18 10:32:36|
|    S1|false|2021-10-18 10:32:46|
|    S1|false|2021-10-18 10:32:56|
|    S1|false|2021-10-18 10:33:06|
+------+-----+-------------------+
only showing top 5 rows



## Let's explore Spark Structured Streaming by example
Please refer to [continuous-analytics-examples/epl_firealarm/readme.md](https://github.com/quantiaconsulting/continuous-analytics-examples/blob/master/epl_firealarm/readme.md) for the EPL version of the following queries.

### Let's create the streaming Data Frames using the data in the kafka topics

In [12]:
raw_streaming_smoke_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", smoke_topic)
  .load())

In [14]:
raw_streaming_smoke_df.isStreaming

True

In [15]:
raw_streaming_smoke_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [16]:
smoke_sdf=(raw_streaming_smoke_df
                      .select(from_json(col("value").cast("string"), smoke_schema).alias("value"))
                      .select("value.*"))

In [17]:
smoke_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- smoke: boolean (nullable = true)
 |-- ts: timestamp (nullable = true)



**NOTE**: it is not a DataFrame, you cannot directly execute an action on it. The following cell *intetionally* gives an error.

In [31]:
smoke_sdf.count()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();;
kafka

In [18]:
temperarture_schema = StructType([
    StructField("sensor", StringType(), True),
    StructField("temperature", DoubleType(), True),
    StructField("ts", TimestampType(), True)])

raw_streaming_temperature_df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", servers)
  .option("startingOffsets", "earliest")
  .option("subscribe", temperature_topic)
  .load())

temperature_sdf = (raw_streaming_temperature_df
                      .select(from_json(col("value").cast("string"), temperarture_schema).alias("value"))
                      .select("value.*"))

In [19]:
temperature_sdf.printSchema()

root
 |-- sensor: string (nullable = true)
 |-- temperature: double (nullable = true)
 |-- ts: timestamp (nullable = true)



### to gain some confidence, let's first inspect the content of the stream of smoke event

In [100]:
basic_query = (smoke_sdf
    .writeStream
    .format("memory") # this is for debug purpose only! DO NOT USE IN PRODUCTION
    .queryName("sinkTable")
    .start())

In [35]:
basic_query.lastProgress

{'id': '32000922-f1e6-4820-8391-94339da67bb3',
 'runId': '1a5eb39a-6ab1-4959-8724-baedc7078cfb',
 'name': 'sinkTable',
 'timestamp': '2021-10-18T10:52:17.994Z',
 'batchId': 29,
 'numInputRows': 1,
 'inputRowsPerSecond': 90.90909090909092,
 'processedRowsPerSecond': 4.405286343612334,
 'durationMs': {'addBatch': 126,
  'getBatch': 0,
  'latestOffset': 1,
  'queryPlanning': 24,
  'triggerExecution': 226,
  'walCommit': 18},
 'stateOperators': [],
 'sources': [{'description': 'KafkaV2[Subscribe[SmokeSensorEvent]]',
   'startOffset': {'SmokeSensorEvent': {'0': 119}},
   'endOffset': {'SmokeSensorEvent': {'0': 120}},
   'numInputRows': 1,
   'inputRowsPerSecond': 90.90909090909092,
   'processedRowsPerSecond': 4.405286343612334}],
 'sink': {'description': 'MemorySink', 'numOutputRows': 1}}

run the following cell to see the most recent content of the sinkTable

In [42]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+-----+-------------------+
|sensor|smoke|                 ts|
+------+-----+-------------------+
|    S1|false|2021-10-18 12:08:22|
|    S1|false|2021-10-18 12:08:12|
|    S1|false|2021-10-18 12:08:02|
|    S1|false|2021-10-18 12:07:52|
|    S1|false|2021-10-18 12:07:42|
+------+-----+-------------------+
only showing top 5 rows



do not forget to stop queries that you are not using

In [101]:
basic_query.stop()

## Q0 - Filter

the temperature events whose temperature is greater than 20 °C (was 50 °C in EPL)

### the SQL style

In [53]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent")

# write your query in SQL, register it and start it
q0 = (spark.sql("select * from TemperatureSensorEvent where temperature > 20")
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the execution plan, we will compare it with cells down with the one of the query in Data Frame style

In [54]:
q0.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@23d5689
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).sensor AS sensor#195, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).temperature AS temperature#196, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).ts AS ts#197]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).temperature > 20.0)
      +- *(1) Project [key#179, value#180, topic#181, partition#182, offset#183L, timestamp#184, timestam

In [44]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+------------------+-------------------+
|sensor|       temperature|                 ts|
+------+------------------+-------------------+
|    S1|22.179079291710003|2021-10-18 12:11:14|
|    S1| 20.29315824114893|2021-10-18 12:11:04|
|    S1|21.832024622651204|2021-10-18 12:10:34|
|    S1| 21.75734312250273|2021-10-18 12:10:24|
|    S1| 20.40437843675412|2021-10-18 12:10:14|
+------+------------------+-------------------+
only showing top 5 rows



In [55]:
# clean up
q0.stop()
spark.catalog.dropTempView("TemperatureSensorEvent")

### The DataFrame style

In [50]:
q0bis = (temperature_sdf
                     .where("temperature > 20") # you can add anything that fits in a SQL where statemente 
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

let's ask for the explanation of the plan. Comparing with the one of the SQL style, you can see that there is no difference. This is expected because the [catalyst optimizer](https://databricks.com/glossary/catalyst-optimizer) created it out of our declarations (which are semantically equivalent)

In [51]:
q0bis.explain()

== Physical Plan ==
WriteToDataSourceV2 org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@43513fd9
+- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).sensor AS sensor#195, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).temperature AS temperature#196, from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).ts AS ts#197]
   +- Filter (from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)).temperature > 20.0)
      +- *(1) Project [key#179, value#180, topic#181, partition#182, offset#183L, timestamp#184, timesta

In [48]:
spark.sql("SELECT * FROM sinkTable ORDER BY TS DESC").show(5)

+------+------------------+-------------------+
|sensor|       temperature|                 ts|
+------+------------------+-------------------+
|    S1| 20.45088665963757|2021-10-18 12:12:24|
|    S1|22.179079291710003|2021-10-18 12:11:14|
|    S1| 20.29315824114893|2021-10-18 12:11:04|
|    S1|21.832024622651204|2021-10-18 12:10:34|
|    S1| 21.75734312250273|2021-10-18 12:10:24|
+------+------------------+-------------------+
only showing top 5 rows



In [52]:
q0bis.stop()

> NOTE: there was no need to
> * create a logic table on top of the streaming data frame
> * drop such a logic table

## Q1 - Avg

the average of all the temperature observation for each sensor up to the last event received

### the SQL sytyle

In [56]:
# create a logic table on top of the streaming data frame
temperature_sdf.createTempView("TemperatureSensorEvent") # this time we will not clean it up, because we use it in the next queries

**NOTE**: the following query gives *intentionally* an error

In [57]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

AnalysisException: Append output mode not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark;;
Aggregate [SENSOR#195], [SENSOR#195, avg(temperature#196) AS avg(temperature)#1173]
+- SubqueryAlias temperaturesensorevent
   +- Project [value#193.sensor AS sensor#195, value#193.temperature AS temperature#196, value#193.ts AS ts#197]
      +- Project [from_json(StructField(sensor,StringType,true), StructField(temperature,DoubleType,true), StructField(ts,TimestampType,true), cast(value#180 as string), Some(Etc/UTC)) AS value#193]
         +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@62b2bce5, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@6047cf1e, org.apache.spark.sql.util.CaseInsensitiveStringMap@24f684c4, [key#179, value#180, topic#181, partition#182, offset#183L, timestamp#184, timestampType#185], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@612ef3f5,kafka,List(),None,List(

The **append output mode** (i.e., the default one) is not supported when there are streaming aggregations on streaming DataFrames/DataSets without watermark, we need to use the **complete output mode**.

In [58]:
query_string = """
SELECT SENSOR, AVG(temperature) 
FROM TemperatureSensorEvent
GROUP BY SENSOR
"""

# write your query in SQL, register it and start it
q1 = (spark.sql(query_string)
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # <-- CHANGE HERE
                     .queryName("sinkTable")
                     .start())

In [61]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show(5) # without ORDER BY TS DESC because the result in the table is already only the most recent

+------+-----------------+
|SENSOR| avg(temperature)|
+------+-----------------+
|    S1|19.98557651555871|
+------+-----------------+



In [62]:
# clean up
q1.stop()

### The DataFrame style

In [63]:
# write your query in SQL, register it and start it
q1bis = (temperature_sdf#.withWatermark("ts", "10 seconds") 
                     .groupBy("sensor")
                     .avg()
                     .writeStream
                     .format("memory")
                     .outputMode("complete") # 
                     .queryName("sinkTable")
                     .start())

In [66]:
# look up the most recent results
spark.sql("SELECT * FROM sinkTable").show(5) # woithout ORDER BY TS DESC because the result in the table is already only the most recent

+------+------------------+
|sensor|  avg(temperature)|
+------+------------------+
|    S1|19.971543405362056|
+------+------------------+



In [67]:
# clean up
q1bis.stop()

## Q2 - Logical Sliding Window

The average temperature observed by each sensor in the last 4 seconds

MEMO: the average should change as soon as the receive a new event

**Not supported**

## Q3 - Logical Tumbling Window

The average temperature of the last 30 seconds every 30 seconds (was 4 seconds in EPL)

NOTE: this query is not possibile in the SQL style

In [68]:
q3 = (temperature_sdf
                         .withWatermark("TS", "30 seconds")
                         .groupBy(window("TS", "30 seconds"),"SENSOR")
                         .avg("TEMPERATURE")
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

In [70]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(5,False) # NOTE: here we order by window instead of ordering by timestamp# window instead of timestamp, again

+------------------------------------------+------+------------------+
|window                                    |SENSOR|avg(TEMPERATURE)  |
+------------------------------------------+------+------------------+
|[2021-10-18 12:23:30, 2021-10-18 12:24:00]|S1    |19.42756865944047 |
|[2021-10-18 12:23:00, 2021-10-18 12:23:30]|S1    |20.786424195844763|
|[2021-10-18 12:22:30, 2021-10-18 12:23:00]|S1    |18.306454897202062|
|[2021-10-18 12:22:00, 2021-10-18 12:22:30]|S1    |19.751798320399647|
|[2021-10-18 12:21:30, 2021-10-18 12:22:00]|S1    |20.13249798474993 |
+------------------------------------------+------+------------------+
only showing top 5 rows



In [71]:
q3.stop()

## Q4 - Physical Sliding Window

The moving average of the last 4 temperature events

**Not supported**

## Q5 - Physical Tumbling Window

The moving average of the last 4 temperature events every 4 events 

**Not supported**

## Q6 - Logical Hopping Window

The average temperature of the last 1 minute (was 4 seconds in EPL) every 5 seconds (was 2 seconds in EPL)

In [72]:
q6 = (temperature_sdf
      .withWatermark("TS", "1 minutes")
      .groupBy(window("TS", "1 minutes", "5 seconds"),"SENSOR")
      .avg("TEMPERATURE")
      .writeStream
      .format("memory")
      .queryName("sinkTable")
      .start())

In [75]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(6,False) # NOTE: here we order by window instead of ordering by timestamp

+------------------------------------------+------+------------------+
|window                                    |SENSOR|avg(TEMPERATURE)  |
+------------------------------------------+------+------------------+
|[2021-10-18 12:25:35, 2021-10-18 12:26:35]|S1    |20.066321580129678|
|[2021-10-18 12:25:30, 2021-10-18 12:26:30]|S1    |20.066321580129678|
|[2021-10-18 12:25:25, 2021-10-18 12:26:25]|S1    |19.79773853431053 |
|[2021-10-18 12:25:20, 2021-10-18 12:26:20]|S1    |19.79773853431053 |
|[2021-10-18 12:25:15, 2021-10-18 12:26:15]|S1    |19.802156398351276|
|[2021-10-18 12:25:10, 2021-10-18 12:26:10]|S1    |19.802156398351276|
+------------------------------------------+------+------------------+
only showing top 6 rows



note the duplicates. They are present because the query is evalauted every 5 seconds, but a new event arrives every 10 seconds.

In [106]:
q6.stop()

## Q7 - Stream-to-Stream Join

In EPL, at this point we moved on to the pattern matching part required to satisfy the information need, i.e., "find every smoke event followed by a temperature event whose temperature is above 50 °C within 2 minutes."

Spark Structured Streaming does not support the EPL's operator `->` (that reads as *followed by*. We need to use a stream-to-stream join.

Let's apply the watermarks on event-time columns and the other two filters.

In [77]:
last_minute_smoke_events = (smoke_sdf
                .withWatermark("ts", "1 minute")
                .where("smoke = True")
                .withColumnRenamed("sensor","sensorSmoke")
                .withColumnRenamed("ts","tsSmoke")
               )

last_minute_high_temperature_events = (temperature_sdf
                .withWatermark("ts", "1 minute")
                .where("temperature > 50")
                .withColumnRenamed("sensor","sensorTemp")
                .withColumnRenamed("ts","tsTemp")
               )

Join with event-time constraints

In [78]:
join_sdf = (last_minute_smoke_events.join(
  last_minute_high_temperature_events, expr("""
    (sensorTemp == sensorSmoke) AND
    (tsTemp > tsSmoke ) AND
    (tsTemp < tsSmoke + interval 2 minute )
    """
    )))

In [79]:
q7 = (join_sdf
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable")
                     .start())

**IMPORTANT** To detect fire, run the appropriate cells in the data generators.

In [90]:
spark.sql("SELECT * FROM sinkTable ORDER BY tsTemp DESC").show(20,False) # note, I change ts in tsTemp

+-----------+-----+-------------------+----------+------------------+-------------------+
|sensorSmoke|smoke|tsSmoke            |sensorTemp|temperature       |tsTemp             |
+-----------+-----+-------------------+----------+------------------+-------------------+
|S1         |true |2021-10-18 12:41:18|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:42:18|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:41:58|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:42:08|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:41:08|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:40:28|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:41:38|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1         |true |2021-10-18 12:41:48|S1        |55.405213828949535|2021-10-18 12:42:20|
|S1       

let's have a look to the progresses

In [88]:
from IPython.display import clear_output
import json
while True:
    print(json.dumps(q7.lastProgress, indent=4))
    print(q7.status)
    time.sleep(1)
    clear_output(wait=True)
    

{
    "id": "ab342e77-f6c5-426f-930e-ba6425fc27d7",
    "runId": "1a8522bc-7780-44fb-95cb-ece8f5b1051b",
    "name": "sinkTable",
    "timestamp": "2021-10-18T12:40:24.208Z",
    "batchId": 72,
    "numInputRows": 0,
    "inputRowsPerSecond": 0.0,
    "processedRowsPerSecond": 0.0,
    "durationMs": {
        "addBatch": 3801,
        "getBatch": 0,
        "latestOffset": 1,
        "queryPlanning": 107,
        "triggerExecution": 3937,
        "walCommit": 15
    },
    "eventTime": {
        "watermark": "2021-10-18T12:39:18.000Z"
    },
    "stateOperators": [
        {
            "numRowsTotal": 25,
            "numRowsUpdated": 0,
            "memoryUsedBytes": 334768,
            "customMetrics": {
                "loadedMapCacheHitCount": 28800,
                "loadedMapCacheMissCount": 0,
                "stateOnCurrentVersionSizeBytes": 41168
            }
        }
    ],
    "sources": [
        {
            "description": "KafkaV2[Subscribe[SmokeSensorEvent]]",
       

KeyboardInterrupt: 

to interrupt the execution of the cell, prese the square icon in the bar or choose *interrupt kernel* from the *kernel* dropdown menu

#### Discussion

> This query is equivalent to the EPL pattern `every a = SmokeSensorEvent(smoke=true) -> every TemperatureSensorEvent(temperature > 50, sensor=a.sensor) where timer:within(1 min)`. 
>
> Do not expect the same performances! It is evaluated as a relational join. Spark Structured Streaming lacks the specilized data structure of Esper.
>
> **It does not tame the torrent effect**, but this is expected! 
>
> Spark Structured Streaming is a Data Stream Management System meant to tame *flow that you cannot stop*

Counterintuitively, we can stop q7 because we only need the streaming Data Frame `join_sdf`. We do not need q7 to write its result in the in memory table.

In [91]:
q7.stop()

## Q8 - Count FireEvent

we are very close to the solution of the running example, we "just" need to count the number of events generated by the previous query over an hopping window of 1 minutes that slides every 30 seconds (was a sliding window of 10 secondsin EPL). 

So let's count the results of Q7. 

In [92]:
q8 = (join_sdf
                     .withWatermark("tsTemp", "1 minutes")
                     .groupBy(window("tsTemp", "1 minutes", "30 seconds"),"sensorTemp")
                     .count()
                     .writeStream
                     .format("memory")
                     .queryName("sinkTable") 
                     .start())

In [98]:
spark.sql("SELECT * FROM sinkTable ORDER BY window DESC").show(100,False)

+------------------------------------------+----------+-----+
|window                                    |sensorTemp|count|
+------------------------------------------+----------+-----+
|[2021-10-18 12:48:30, 2021-10-18 12:49:30]|S1        |72   |
|[2021-10-18 12:48:00, 2021-10-18 12:49:00]|S1        |72   |
|[2021-10-18 12:47:30, 2021-10-18 12:48:30]|S1        |72   |
|[2021-10-18 12:47:00, 2021-10-18 12:48:00]|S1        |72   |
|[2021-10-18 12:46:30, 2021-10-18 12:47:30]|S1        |72   |
|[2021-10-18 12:46:00, 2021-10-18 12:47:00]|S1        |72   |
|[2021-10-18 12:45:30, 2021-10-18 12:46:30]|S1        |72   |
|[2021-10-18 12:45:00, 2021-10-18 12:46:00]|S1        |72   |
|[2021-10-18 12:44:30, 2021-10-18 12:45:30]|S1        |72   |
|[2021-10-18 12:44:00, 2021-10-18 12:45:00]|S1        |72   |
|[2021-10-18 12:43:30, 2021-10-18 12:44:30]|S1        |72   |
|[2021-10-18 12:43:00, 2021-10-18 12:44:00]|S1        |72   |
|[2021-10-18 12:42:30, 2021-10-18 12:43:30]|S1        |72   |
|[2021-1

In [99]:
q8.stop()